In [ ]:
import requests
import os
import numpy as np
from netCDF4 import Dataset
import datetime as dt
import shutil
import pandas as pd

In [ ]:
def change_val_1D_char(ds,varname,new_val):
    var=ds.variables[varname]
    old_val=var[:].tobytes().decode().strip()
    
    print("Modifying ",varname," from ", old_val," to ",new_val)

    new_array=np.chararray(var.shape)
    new_array[:]=" " 
    for i in range(len(new_val)):
        if len(var.shape)==2:
            new_array[0,i]=new_val[i]
        else:
            new_array[i]=new_val[i]
            

    var[:]=new_array

In [ ]:
def change_val_2D_char(ds,varname,dim_level,new_val):
    var=ds.variables[varname]
    old_val=var[dim_level,:].tobytes().decode().strip()
    print("Modifying ",varname," for dim_level ", dim_level, "from ", old_val," to ",new_val)

    new_type_array=np.chararray(var[dim_level,:].shape)
    new_type_array[:]=" " 
    for i in range(len(new_val)):
        new_type_array[i]=new_val[i]
    var[dim_level,:]=new_type_array[:]

In [ ]:
dac="coriolis"
liste_wmos=['1901351']

file_prefix='nemo_0100'
in_positions_file="C:/Users/ddobler/Documents/NEMO/NEMO_profile_files/co010103v1/parsed_pos_dot_profile/pos_dot_profile_"+file_prefix+".csv"
print(in_positions_file,os.path.exists(in_positions_file))
if os.path.exists(in_positions_file):
    coordinates = pd.read_csv(in_positions_file,sep=";")
else:
    print(in_positions_file, " does not exist, WARNING !")
CYCLE_NUMBER=(np.array(coordinates.CYCLE)).astype('int')
LATITUDE=np.array(coordinates.LATITUDE)
LONGITUDE=np.array(coordinates.LONGITUDE)
JULD=np.array(coordinates.JULD,dtype=np.datetime64)
JULD=((JULD-np.datetime64("1950-01-01T00:00:00")).astype('int'))/86400
JULD_QC=np.array(coordinates.JULD_QC)
JULD_LOCATION=JULD[:]
POSITION_QC=np.array(coordinates.POSITION_QC)


in_dc_reference_file="C:/Users/ddobler/Documents/NEMO/NEMO_profile_files/co010103v1/p_coriolis_db_reference/dc_reference_for_"+file_prefix+".csv"
print("loading dc_reference - extracted beforehand from coriolis db ")
print(in_dc_reference_file,os.path.exists(in_dc_reference_file))
if os.path.exists(in_dc_reference_file):
    dc_reference_from_coriolis = pd.read_csv(in_dc_reference_file,sep=";",dtype={'wmo': str, 'cycle': int, 'dir': str, 'dc_reference': str})
else:
    print(in_dc_reference_file, " does not exist, WARNING !")

cycle_start=61
cycle_end=69
data_mode_file="D"



In [ ]:
wrk_dir="C:/Users/ddobler/Documents/08_DD_scripts/01_Scripts_Metadata_and_Data_Update/ProfData_WorkDir/lot_002/"

#for wmo in list_wmo:
for iwmo in range(len(liste_wmos)):
#for iwmo in range(3):

    wmo=liste_wmos[iwmo]
    #nnn_val=liste_val[iwmo]
    #nnn_val2=liste_val2[iwmo]
    #nnn_val3=liste_val3[iwmo]
    #nnn_val4=liste_val4[iwmo]
    #nnn_val5=liste_val5[iwmo]

    history_update_text='updated by D.Dobler (Euro-Argo ERIC) for a JULD error of approx. 1day'

    # Select wmo to treat
    #wmo="6900203"
    print("\n\n---------------------------")
    print("Treating wmo ",wmo)


    for icycle in range(cycle_start,cycle_end+1):
    # select modifications to perform

        print("extracting dc_reference for the current cycle")
        l_dc_ref=" "
        try:
            dc_reference_from_coriolis_icycle=dc_reference_from_coriolis.loc[(dc_reference_from_coriolis['cycle'] == icycle) & 
                                                                             (dc_reference_from_coriolis['dir'] == 'A') & 
                                                                             (dc_reference_from_coriolis['wmo'] == wmo)]
            l_dc_ref=str(np.array(dc_reference_from_coriolis_icycle['dc_reference'])[0])
        except:
            print("No DC_REFERENCE found for this cycle")
        print('DC_REFERENCE=',l_dc_ref)

        if icycle == 15:
            continue

        i_new_juld                    = 0
        new_juld = JULD[np.where(CYCLE_NUMBER==icycle)]
    
        i_new_PI_NAME                 = 0
        new_PI_NAME="Birgit KLEIN"
    
        i_new_pf_type                 = 0
        new_pf_type='NEMO'
    
        i_new_positioning_system_1    = 0
        new_positioning_system_1='GPS'
    
        i_new_float_sn                = 0
        new_float_sn="tutu"

        i_new_dc_reference            = 1
        new_dc_reference  = l_dc_ref
    
        i_history_update_3_1          = 0
    
        i_ftp_from_ifremer            = 1
        #where to save
        
        if not os.path.exists(wrk_dir):
            os.mkdir(wrk_dir)
        file_to_update=data_mode_file + wmo + "_{0:03d}.nc".format(icycle)
        OUT_FILE=wrk_dir + file_to_update
        print(OUT_FILE)
        
     
        # where to get 
        if i_ftp_from_ifremer:
            print("Getting file from data-argo.ifremer.fr")
            URL = "https://data-argo.ifremer.fr/dac/"+dac+"/"+wmo+"/profiles/"+data_mode_file + wmo + "_{0:03d}.nc".format(icycle)
            #URL = "https://data-argo.ifremer.fr/dac/"+dac+"/"+wmo+"/profiles/"+ file_to_update
            print(URL)
            # commands
            response = requests.get(URL)
            open(OUT_FILE, "wb").write(response.content)
        else:
            print("Getting file from local directory")
            #IN_FILE="C:/Users/ddobler/Documents/09_Scripts_WD/lot_014_deleted_level_in/" + wmo + "_meta.nc"
            shutil.copyfile(IN_FILE,OUT_FILE)
    
        #print("Making changes for wmo ",wmo)
    
        ds=Dataset(OUT_FILE,'a')
    
        # update the global attribute history:
        date_courante=dt.datetime.now(dt.timezone.utc)
        date_courante_str=date_courante.strftime("%Y-%m-%dT%H:%M:%SZ")
        #lhistory=date_courante_str+ ' creation; '+date_courante_str+' last update (coriolis float real time data processing)'
    
    
        if i_new_positioning_system_1:
            change_val_2D_char(ds,"POSITIONING_SYSTEM",0,new_positioning_system_1)
    
        if i_history_update_3_1:
            print("Modifying global attribute section")
            ds.title = "Argo float vertical profile"
            ds.institution = "CORIOLIS"
            ds.source = "Argo float"
            ds.history = lhistory
            ds.references = "http://www.argodatamgt.org/Documentation" ;
            ds.user_manual_version = "3.1"
            ds.Conventions = "Argo-3.1 CF-1.6"
            #new_ds.decoder_version = ""
            ds.featureType = "trajectoryProfile"
    
    
        if i_new_juld:
            l_juld=ds.variables["JULD"]
            print("Modifying JULD from ", l_juld[:]," to ",new_juld)
            l_juld[:]=new_juld
            l_juld_loc=ds.variables["JULD_LOCATION"]
            print("Modifying JULD_LOCATION from ", l_juld_loc[:]," to ",new_juld)
            l_juld_loc[:]=new_juld
            
            
        if i_new_float_sn: change_val_1D_char(ds,"FLOAT_SERIAL_NO",new_float_sn)
        if i_new_dc_reference: 
            print(new_dc_reference)
            
            change_val_1D_char(ds,"DC_REFERENCE",new_dc_reference)
                    
        if i_new_pf_type:
            try:
                varname="PLATFORM_TYPE"
                var=ds.variables[varname]
            except:
                varname="PLATFORM_MODEL"
                var=ds.variables[varname]
            change_val_1D_char(ds,varname,new_pf_type)     
            
        
        if i_new_PI_NAME: change_val_1D_char(ds,"PI_NAME",new_PI_NAME)
    
        
        
        print("\n Updating history global attribute")
    
        tmp=str(ds.variables["DATE_CREATION"][:].tobytes(),'utf-8')
        record_date_creation=tmp[:4]+"-"+tmp[4:6]+"-"+tmp[6:8]+"T"+tmp[8:10]+":"+tmp[10:12]+":"+tmp[12:14]+"Z"
        #print(record_date_creation)
    
    
        # update the global attribute history:
        date_courante=dt.datetime.now(dt.timezone.utc)
        date_courante_str=date_courante.strftime("%Y-%m-%dT%H:%M:%SZ")
        lhistory=record_date_creation+ ' creation; '+date_courante_str + ' ' + history_update_text 
        #lhistory=record_date_creation+ ' creation; '+date_courante_str+' updated'
        try:
            print("Old history global attr: ",ds.history)
        except:
            print("Old history global attr did not exist")
        ds.history = lhistory
        print("New history global attr: ",ds.history)
    
        print("Updating DATE_UPDATE")
        varname="DATE_UPDATE"
        new_val=date_courante.strftime("%Y%m%d%H%M%S")
        change_val_1D_char(ds,varname,new_val)

    
        ds.close()

In [ ]:
print("wmo=",wmo)

# check step
ds=Dataset(OUT_FILE,'r')
SENSOR_TYPE=ds.variables["SENSOR"]
SENSOR_MAKER=ds.variables["SENSOR_MAKER"]
SENSOR_MODEL=ds.variables["SENSOR_MODEL"]
SENSOR_SN=ds.variables["SENSOR_SERIAL_NO"]
DATE_UPDATE=ds.variables["DATE_UPDATE"]
LAUNCH_LATITUDE=ds.variables["LAUNCH_LATITUDE"]
LAUNCH_LONGITUDE=ds.variables["LAUNCH_LONGITUDE"]
LAUNCH_DATE=ds.variables["LAUNCH_DATE"]
START_DATE=ds.variables["START_DATE"]
PLATFORM_MAKER=ds.variables["PLATFORM_MAKER"]

hh=ds.history

print("\n--global attribute history")
print(hh)

print("\n--PLATFORM_MAKER:")
print(PLATFORM_MAKER[:].tobytes().decode().strip())

print("\n--SENSOR_TYPE:")
print(SENSOR_TYPE[:,:].tobytes().decode().strip())

print("\n--SENSOR_MAKER:")
print(SENSOR_MAKER[:,:].tobytes().decode().strip())

print("\n--SENSOR_MODEL:")
print(SENSOR_MODEL[:,:].tobytes().decode().strip())

print("\n--SENSOR_SN:")
print(SENSOR_SN[:,:].tobytes().decode().strip())

print("\n--LAUNCH_LONGITUDE:")
print(LAUNCH_LONGITUDE[:])

print("\n--LAUNCH_LATITUDE:")
print(LAUNCH_LATITUDE[:])

print("\n--LAUNCH_DATE:")
print(LAUNCH_DATE[:].tobytes().decode().strip())

print("\n--START_DATE:")
print(START_DATE[:].tobytes().decode().strip())

print("\n--DATE_UPDATE:")
print(DATE_UPDATE[:].tobytes().decode().strip())

ds.close()